In [52]:
import time
import logging
import json
from substrateinterface import SubstrateInterface

class BlockMonitor:
    def __init__(self, substrate, logger, verbose=False):
        self.substrate = substrate
        self.logger = logger
        self.verbose = verbose  # If True, print full raw extrinsic and event data.
        self.should_exit = False  # Flag to stop monitoring
        # Dictionary to store pending stake extrinsic calls.
        # Keys: unique call IDs; Values: dict with call details.
        self.pending_calls = {}
        self.call_counter = 0  # Unique counter for pending calls

    def _get_block_hash(self, block_num):
        """Retrieve the block hash for a given block number."""
        try:
            block_hash = self.substrate.get_block_hash(block_num)
            return block_hash
        except Exception as e:
            self.logger.error(f"Error getting block hash for block {block_num}: {e}")
            return None

    def _get_extrinsics(self, block_hash):
        """Retrieve all extrinsics for a given block hash."""
        try:
            block = self.substrate.get_block(block_hash=block_hash)
            return block.get('extrinsics', [])
        except Exception as e:
            self.logger.error(f"Error getting extrinsics for block {block_hash}: {e}")
            return []

    def _process_extrinsics(self, block_num, extrinsics):
        """Process extrinsics in a block to record pending stake-related calls."""
        for ext in extrinsics:
            try:
                # Use ext.value if available; otherwise assume ext is a dict.
                ext_dict = ext.value if hasattr(ext, 'value') else ext

                if self.verbose:
                    self.logger.info("Raw extrinsic data: %s", json.dumps(ext_dict, indent=2, default=str))
                
                # Extract the coldkey from the extrinsic's "address" field.
                coldkey = ext_dict.get('address')
                
                call_info = ext_dict.get('call', {})
                call_module = call_info.get('call_module')
                call_function = call_info.get('call_function')
                # Only process stake-related calls from SubtensorModule.
                if call_module == 'SubtensorModule' and call_function in [
                    'add_stake', 'remove_stake', 'add_stake_limit', 'remove_stake_limit'
                ]:
                    params = call_info.get('call_args', [])
                    hotkey = None
                    call_amount = None  # The call amount as submitted in the extrinsic.
                    netuid_param = None
                    for param in params:
                        pname = param.get('name')
                        if pname == 'hotkey':
                            hotkey = param.get('value')
                        elif pname in ['amount', 'amount_staked', 'amount_unstaked']:
                            try:
                                call_amount = float(param.get('value', 0)) / 1e9
                            except Exception as conv_e:
                                self.logger.error(f"Error converting call amount from {param.get('value')}: {conv_e}")
                        elif pname == 'netuid':
                            try:
                                netuid_param = int(param.get('value', -1))
                            except Exception as conv_e:
                                self.logger.error(f"Error converting netuid: {conv_e}")
                    if call_amount is None:
                        self.logger.info(f"Call args for {call_function}: {json.dumps(params, indent=2, default=str)}")
                    
                    call_id = self.call_counter
                    self.call_counter += 1
                    self.pending_calls[call_id] = {
                        'block_num': block_num,
                        'call_function': call_function,
                        'coldkey': coldkey,      # The extrinsic signer (coldkey)
                        'hotkey': hotkey,
                        'netuid': netuid_param,
                        'call_amount': call_amount,  # The amount as submitted in the extrinsic.
                        'final_amount': None,        # To be updated based on event.
                        'validated': False,
                        'raw': call_info            # Full raw call info for debugging.
                    }
                    self.logger.info(
                        f"Recorded pending call {call_id} in block {block_num}: {call_function} - "
                        f"Coldkey: {coldkey}, Hotkey: {hotkey}, Call Amount: {call_amount if call_amount is not None else 'None'}, "
                        f"Netuid: {netuid_param}"
                    )
            except Exception as e:
                self.logger.error(f"Error processing extrinsic in block {block_num}: {e}")

    def _process_events(self, block_num, events_decoded):
        """Process events in a block and attempt to match them with pending extrinsic calls."""
        for idx, event_record in enumerate(events_decoded.value):
            try:
                if self.verbose:
                    self.logger.info("Raw event record %s: %s", idx, json.dumps(event_record, indent=2, default=str))
                
                event = event_record.get('event', {})
                module_id = event.get('module_id')
                event_id = event.get('event_id')
                attributes = event.get('attributes')
                
                # We care about stake events from SubtensorModule.
                if module_id == 'SubtensorModule' and event_id in ['StakeAdded', 'StakeRemoved']:
                    event_hotkey = None
                    event_netuid = None
                    event_amount = None
                    # Assume attributes is a list/tuple with at least 5 items:
                    # [caller, hotkey, amount, new_total, netuid]
                    if isinstance(attributes, (list, tuple)) and len(attributes) >= 5:
                        event_hotkey = attributes[1]
                        try:
                            event_amount = float(attributes[2]) / 1e9
                        except Exception as conv_e:
                            self.logger.error(f"Error converting event amount: {conv_e}")
                        try:
                            event_netuid = int(attributes[4])
                        except Exception as conv_e:
                            self.logger.error(f"Error converting event netuid: {conv_e}")
                    else:
                        self.logger.info(f"Event {idx} attributes not in expected format: {attributes}")
                    
                    self.logger.info(
                        f"Event {idx} in block {block_num}: {event_id} - Hotkey: {event_hotkey}, "
                        f"Event Amount: {event_amount if event_amount is not None else 'None'}, Netuid: {event_netuid}"
                    )
                    
                    # Mapping: for add_stake & add_stake_limit, expect StakeAdded; for remove_stake & remove_stake_limit, expect StakeRemoved.
                    mapping = {
                        'add_stake': 'StakeAdded',
                        'add_stake_limit': 'StakeAdded',
                        'remove_stake': 'StakeRemoved',
                        'remove_stake_limit': 'StakeRemoved'
                    }
                    for call_id, call_data in list(self.pending_calls.items()):
                        expected_event_id = mapping.get(call_data['call_function'])
                        if expected_event_id != event_id:
                            continue
                        if call_data['hotkey'] != event_hotkey or call_data['netuid'] != event_netuid:
                            continue
                        # For stake orders, record final amount from the event; for unstake orders, use the call amount.
                        if call_data['call_function'] in ['add_stake', 'add_stake_limit']:
                            final_amount = event_amount
                        else:
                            final_amount = call_data['call_amount']
                        call_data['final_amount'] = final_amount
                        fee = None
                        if call_data['call_function'] in ['add_stake', 'add_stake_limit'] and call_data['call_amount'] is not None and event_amount is not None:
                            fee = call_data['call_amount'] - event_amount
                        self.logger.info(
                            f"Validated pending call {call_id} (from block {call_data['block_num']}) with event "
                            f"{event_id} in block {block_num}: Coldkey: {call_data['coldkey']}, Hotkey: {event_hotkey}, "
                            f"Final Amount: {final_amount:.9f}"
                            f"{', Fee: ' + str(fee) if fee is not None else ''}, Netuid: {event_netuid}"
                        )
                        del self.pending_calls[call_id]
            except Exception as e:
                self.logger.error(f"Error processing event {idx} in block {block_num}: {e}")

    def _purge_old_calls(self, current_block):
        """Purge pending extrinsic calls older than a threshold.
           Standard stake calls are purged after 20 blocks.
           Limit orders (add_stake_limit, remove_stake_limit) are retained for 216000 blocks (~1 month).
        """
        to_delete = []
        for call_id, call_data in self.pending_calls.items():
            if call_data['call_function'] in ['add_stake_limit', 'remove_stake_limit']:
                threshold = 216000
            else:
                threshold = 20
            if current_block - call_data['block_num'] >= threshold:
                self.logger.info(f"Discarding pending call {call_id} from block {call_data['block_num']} (older than {threshold} blocks)")
                to_delete.append(call_id)
        for call_id in to_delete:
            del self.pending_calls[call_id]

    def block_header_handler(self, block_header, update_nr, subscription_id):
        """Callback for new block headers. Processes extrinsics and events and purges old pending calls."""
        self.logger.info("Block header received: %s", json.dumps(block_header, indent=2, default=str))
        try:
            block_num_val = block_header.get("header", {}).get("number")
            block_num = int(block_num_val, 16) if isinstance(block_num_val, str) else block_num_val
            self.logger.info("Extracted block number: %s", block_num)
            block_hash = self.substrate.get_block_hash(block_num)
            if not block_hash:
                self.logger.error("Could not compute block hash for block number %s", block_num)
                return
            self.logger.info("Computed block hash for block %s: %s", block_num, block_hash)
        except Exception as e:
            self.logger.error("Error computing block hash from header: %s", e)
            return

        extrinsics = self._get_extrinsics(block_hash)
        self._process_extrinsics(block_num, extrinsics)

        try:
            events_decoded = self.substrate.query("System", "Events", block_hash=block_hash)
            self.logger.info("Processing events for block %s", block_num)
            self._process_events(block_num, events_decoded)
        except Exception as e:
            self.logger.error(f"Error querying events for block {block_hash}: {e}")

        self._purge_old_calls(block_num)

    def start_monitoring(self):
        self.logger.info(f"Starting block monitoring at {self.substrate.url}")
        subscription = self.substrate.subscribe_block_headers(self.block_header_handler)
        try:
            while not self.should_exit:
                time.sleep(1)
        except KeyboardInterrupt:
            self.logger.info("Keyboard interrupt detected, stopping monitoring.")
            self.should_exit = True
        subscription.unsubscribe()
        self.logger.info("Block monitoring stopped")

# Example usage:
if __name__ == "__main__":
    logging.basicConfig(level=logging.INFO)
    logger = logging.getLogger("BlockMonitor")
    
    substrate = SubstrateInterface(
        url="wss://entrypoint-finney.opentensor.ai:443"
    )
    # Set verbose=True to print full raw extrinsic and event data.
    monitor = BlockMonitor(substrate, logger, verbose=False)
    monitor.start_monitoring()


INFO:BlockMonitor:Starting block monitoring at wss://entrypoint-finney.opentensor.ai:443
INFO:BlockMonitor:Block header received: {
  "header": {
    "parentHash": "0x2f801277b54f3d015b6ac0a5322487dcb3eebb260af5e998799b5a609e809687",
    "number": 5057195,
    "stateRoot": "0x9e9989d1147c4bad764b344ae6606004aea7df9423fdcfed4e4ab99d8b52d6e5",
    "extrinsicsRoot": "0x092294a3c5cfe4dd740ee4757cfddd6690b4f1bd6d68c83da71cd4bf5a4f85e3",
    "digest": {
      "logs": [
        "{'PreRuntime': ('0x61757261', '0x83f4a50800000000')}",
        "{'Consensus': ('0x66726f6e', '0x010c03cdc726b443a3af6f56bef9fb5be70d668c70d9367ad0c9579857e1bdc1b900')}",
        "{'Seal': ('0x61757261', '0x2ce3a4c09f612573db32adea6fcee2932dc568441c6a378afcd66ede20a3a564ea551cf370a72854b4247c7d5df215c98ddeb04ab1d82cbbaf05e1dc5c718a89')}"
      ]
    }
  }
}
INFO:BlockMonitor:Extracted block number: 5057195
INFO:BlockMonitor:Computed block hash for block 5057195: 0x511958f56fffb4cd0ed8fa91b252bdd980fbb49c5af633c1c889d88

KeyboardInterrupt: 

In [29]:
from substrateinterface import SubstrateInterface

# Initialize the Substrate interface.
substrate = SubstrateInterface(
    url="wss://entrypoint-finney.opentensor.ai:443"
)

def block_header_handler(block_header, update_nr, subscription_id):
    # Extract the block hash from the header.
    block_hash = block_header.get("header", {}).get("hash")
    print("\nNew block received with hash:", block_hash)
    try:
        # Query the decoded events for this block.
        events_decoded = substrate.query("System", "Events", block_hash=block_hash)
        print("Decoded System.Events for block", block_hash, ":")
        for idx, event_record in enumerate(events_decoded.value):
            print(f"Event {idx}: {event_record}")
    except Exception as e:
        print("Error querying events for block", block_hash, ":", e)

# Subscribe to new block headers.
subscription = substrate.subscribe_block_headers(block_header_handler)



New block received with hash: None
Decoded System.Events for block None :
Event 0: {'phase': 'Initialization', 'extrinsic_idx': None, 'event': {'event_index': '0705', 'module_id': 'SubtensorModule', 'event_id': 'WeightsSet', 'attributes': (41, 39)}, 'event_index': 7, 'module_id': 'SubtensorModule', 'event_id': 'WeightsSet', 'attributes': (41, 39), 'topics': []}
Event 1: {'phase': 'Initialization', 'extrinsic_idx': None, 'event': {'event_index': '0705', 'module_id': 'SubtensorModule', 'event_id': 'WeightsSet', 'attributes': (41, 2)}, 'event_index': 7, 'module_id': 'SubtensorModule', 'event_id': 'WeightsSet', 'attributes': (41, 2), 'topics': []}
Event 2: {'phase': 'Initialization', 'extrinsic_idx': None, 'event': {'event_index': '0705', 'module_id': 'SubtensorModule', 'event_id': 'WeightsSet', 'attributes': (41, 55)}, 'event_index': 7, 'module_id': 'SubtensorModule', 'event_id': 'WeightsSet', 'attributes': (41, 55), 'topics': []}
Event 3: {'phase': 'Initialization', 'extrinsic_idx': Non

KeyboardInterrupt: 